In [26]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer


# Ερώτημα 1

In [27]:
df = pd.read_csv('diabetes.csv')
input_df = df[['Pregnancies', 'BloodPressure', 'BMI', 'Age']]
targets_df = df['Glucose']

# Διόρθωση ακυρων τιμών
input_df = input_df.copy()
input_df['BloodPressure'] = input_df['BloodPressure'].replace(0, np.nan)
input_df['BMI'] = input_df['BMI'].replace(0, np.nan)
imputer = SimpleImputer(strategy='median')
input_df[['BloodPressure', 'BMI']] = imputer.fit_transform(input_df[['BloodPressure', 'BMI']])

# Κανονικοποιηση
input_df_normalized = (input_df - input_df.min()) / (input_df.max() - input_df.min())
x_normalized = input_df_normalized.to_numpy()

x = input_df.to_numpy()
y = targets_df.to_numpy()

seed = 0
test_size = 0.3

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=seed)


# Ερώτημα 2

In [29]:
model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = model.score(x_test, y_test)

print("Mean Squared Error: ", mse)
print("Mean Absolute Error: ", mae)

Mean Squared Error:  918.593768059542
Mean Absolute Error:  23.206084156470755


Η καταλληλοτερη μετρική για την συγκεκριμένη εφαρμογή θεωρώ πως είναι το μέσο τετραγωνικό σφάλμα (MSE), καθώς επηρεάζεται περισσότερο από μεγάλα σφάλματα, που σε μια ιατρική εφαρμογή όπως αυτή θα μπορούσαν να έχουν σοβαρές συνέπειες.